# Procesar los datos de los videos

Este notebook se encarga de transformar los datos crudos provenientes de archivos de video en un formato estructurado y útil para el análisis.

**Objetivo**
*  Leer y procesar los archivos crudos para crear un DataFrame que contenga información esencial sobre cada grabación, incluyendo detalles sobre el paciente, el número de repetición, el ID del movimiento, la precisión del gesto (si es correcto o no), y la posición de los keypoints.
* Crear un segundo DataFrame que contenga los ángulos calculados entre algunos de los keypoints, lo que ayudará a entender mejor la postura y el movimiento durante la ejecución del gesto.

**Entrada**
* Archivos de datos de videos dentro del directorio ``SkeletonData/RawData``, que contienen información sobre el paciente, las repeticiones, los IDs de movimiento, y las posiciones de keypoints.

**Salida**
* ``Resultados/raw_pacientes.csv``: contiene información detallada sobre cada grabación. Es el archivo de entrada de ``analisis_datos.ipynb``.
* ``Resulados/angles.csv``: incluye los ángulos calculados entre keypoints, facilitando el análisis de la postura y el movimiento durante el gesto. Este será el archivo de entrada de ``medidas_angulos.ipynb`` y de ``analisis_datos.ipynb``.

**Índice**
1. [Procesar Raw Data Folder](#1-procesar-raw-data-folder)\
    1.1 [Datos de los nombres de los archivos](#11-datos-de-los-nombres-de-los-archivos)\
    1.2 [Datos de los archivos](#12-datos-de-los-archivos)
2. [Calcular los ángulos](#2-calcular-los-ángulos)


In [1]:
# importar librerias necesarias
import pandas as pd # para manejar dataframes
import main

*****
## 1. Procesar Raw Data Folder

El conjunto de datos incluye a 30 sujetos, de los cuales 14 son sanos y 16 son pacientes. Cada participante realiza 9 tipos de ejercicios, con 6 repeticiones de cada uno.

La carpeta contiene 2598 archivos de texto con información de los videos, donde cada archivo corresponde a un sujeto realizando una repetición de un gesto. Hay información tanto en el contenido del archivo como en el propio nombre del archivo.

* En el nombre del archivo se encuentran los identificadores del sujeto, del día, del gesto, el número de repetición, si es correcto y la posición del sujeto durante el ejercicio.

* Dentro de cada archivo hay información sobre las coordenadas de 25 puntos del cuerpo captados por un sensor que graba a 30 fps. Por lo tanto, por cada segundo de video, se generan 30 líneas en el archivo de texto. Cada línea sigue el siguiente formato: marca de tiempo, XX, XX, seguido de 25 pares de (NombreArticulación, EstadoSeguimiento, coordenada 3D X, coordenada 3D Y, coordenada 3D Z, coordenada 2D X, coordenada 2D Y).


### 1.1 Datos de los nombres de los archivos

Los nombre de los archivos siguen el siguiente formato: *SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt*

* *SubjectID*: id uniquely identifying the person performing the exercise
* *DateID*: id identifying the session in which the person was recorded
* *GestureLabel*: Label identifying the gesture; possible values are from 0 to 8
* *RepetitionNumber*: Each gesture was repeated several times and this shows the repetition number
* *CorrectLabel*: A value of 1 represents a gesture labeled as being correctly executed, while a value of 2 is for a gesture labeled as incorrect
* *Position*: Some of the persons performed the gestures sitting on a chair or wheelchair, while others standing

### 1.2 Datos de los archivos
Each raw data file contains per line: timestamp, XX, XX, followed by a 25 pairs of (JointName, TrackedStatus, 3d coordinate X, 3d coordinate Y, 3d coordinate Z, 2d coordinate X, 2d coordinate Y)

In [4]:
# directorio donde se encuentran los datos
directory = '../dataset/SkeletonData/RawData' 

columnas = ['SubjectID', 'DateID', 'GestureLabel', 'RepetitionNumber', 'CorrectLabel', 'Position',
            'JointName', 'TrackedStatus', '3D_X', '3D_Y', '3D_Z', '2D_X', '2D_Y']

# Extraer la informacion y almacenarla en un DataFrame
df_data = main.leer_datos_archivo(directory, columnas)

In [5]:
# Eliminar columnas innceserarias
df_data.drop(['TrackedStatus', 'DateID', '2D_X', '2D_Y'], axis=1, inplace=True)

# Añadir columna más descriptiva de los gestos
gesture_mapping = {
    '0': 'EFL',
    '1': 'EFR',
    '2': 'SFL',
    '3': 'SFR',
    '4': 'SAL',
    '5': 'SAR',
    '6': 'SFE',
    '7': 'STL',
    '8': 'STR'
}

# Nueva columna basada en el mapeo
columna_gestrure_name = df_data['GestureLabel'].map(gesture_mapping)

# Insertar la columna al lado de GestureLabel
df_data.insert(2, 'GestureName', columna_gestrure_name)

df_data

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,Position,JointName,3D_X,3D_Y,3D_Z
0,101,0,EFL,1,1,stand,SpineBase,-0.1028086,0.06965441,2.464606
1,101,0,EFL,1,1,stand,SpineMid,-0.1026228,0.3837799,2.438919
2,101,0,EFL,1,1,stand,Neck,-0.1025293,0.6877351,2.40196
3,101,0,EFL,1,1,stand,Head,-0.1190992,0.8358598,2.373549
4,101,0,EFL,1,1,stand,ShoulderLeft,-0.2826451,0.5525576,2.38421
...,...,...,...,...,...,...,...,...,...,...
5707520,307,8,STR,9,1,stand,SpineShoulder,-0.05799517,0.5291457,2.422904
5707521,307,8,STR,9,1,stand,HandTipLeft,-0.302538,-0.1131345,2.284269
5707522,307,8,STR,9,1,stand,ThumbLeft,-0.2783904,-0.06298634,2.269769
5707523,307,8,STR,9,1,stand,HandTipRight,0.1525867,-0.136378,2.45287


In [6]:
# guardar los datos en formato csv
df_data.to_csv('../Resultados/raw_pacientes.csv', index=False)

******
## 2. Calcular los ángulos

<div style="text-align: center;">
<img src="../Imagenes/gestures.png" width="500"/>
</div>

In [8]:
# Aplica la función para caluclar los ángulos a los datos en crudo
df_angles = main.apply_angles(df_data)
# Visualizar el dataframe
df_angles

In [11]:
# guardar los datos en formato csv
df_angles.to_csv('../Resultados/angles.csv', index=False)